In [ ]:
import geopandas as gpd
import pandas as pd
import contextily as ctx
import pathlib
import numpy as np
import networkx
from matplotlib import pyplot as plt
from shapely.geometry import Point, Polygon, LineString

# Storm Forecasts & Track
Storm forecasts and historical track will be published for each in-game day.

The forecasts represent model predictions for where the hurricane might be at the given timestamp.

The storm track represents the true path that the storm took in the past.

In [ ]:
# Forecasts
day1_forecast = gpd.read_file('https://files.bwsi-remote-sensing.net/data/final/day1/day1_forecasts_2023.geojson')

In [ ]:
random_color = lambda: np.random.random(3)

In [ ]:
# look at the forecasts
day1_forecast

In [ ]:
# each forecast report has a number of possible hurricane routes
# they are identified by the prediction column
fig, ax = plt.subplots(1, 1, figsize=(10,10))
for pred_id, pred in day1_forecast.groupby('prediction'):
    c = random_color()
    pred.plot(color=c, ax=ax)
    line_to_plot = LineString(pred.geometry.values)
    ax.plot(line_to_plot.xy[0], line_to_plot.xy[1], color=c, label=f'{pred_id}')
ax.legend(title='prediction id')

In [ ]:
# track shows where the storm actually went in the past
day1_track = gpd.read_file('https://files.bwsi-remote-sensing.net/data/final/day1/day1_track_past.geojson')
day1_track.plot()

# Location of Areas of Interest


In [ ]:
!wget https://files.bwsi-remote-sensing.net/data/final/R1-Hospitals.zip -O R1-Hospitals.zip
!unzip R1-Hospitals.zip

In [ ]:
!wget https://files.bwsi-remote-sensing.net/data/final/R1-Shelters.zip -O R1-Shelters.zip
!unzip R1-Shelters.zip

In [ ]:
hospitals_gdf = gpd.read_file('R1-Hospitals/')
shelters_gdf = gpd.read_file('R1-Shelters/')

In [ ]:
hospitals_gdf

In [ ]:
shelters_gdf

# Game Grid
This is the grid that represents the scope of the response, and also the transportation network
It uses the [Military Grid Reference System](https://en.wikipedia.org/wiki/Military_Grid_Reference_System) to divide the world into a grid. There is a unique alphanumeric string that identifies each cell. We are working at the 1km resolution of the grid. See also the [US National Grid](https://www.fgdc.gov/usng/how-to-read-usng) which is nearly identical, except for some [edge-cases](https://www.maptools.com/tutorials/mgrs_usng_diffs).

The `MGRS` column is a unique alphanumeric ID for each cell. It should be used as your node ID for your transport network.

In [ ]:
# downloading pre-hurricane game grid (pretty big file ~ 111 MB)
!wget https://files.bwsi-remote-sensing.net/data/final/game_grid_2022.geojson

In [ ]:
game_grid = gpd.read_file('game_grid_2023.geojson')

In [ ]:
game_grid

## Overall bounds of map
Useful for all teams to find additional GIS and remote sensing resources to use

In [ ]:
w,s,e,n = game_grid.total_bounds
print(w,s,e,n)

## Get the cells neighboring each cell
Buffer and use spatial join with `overlap` to find which cells are neighboring each other

In [ ]:
buffered_game_grid = game_grid.to_crs('epsg:3857')
buffered_game_grid.geometry = buffered_game_grid.buffer(10)
neighboring_gdf = gpd.sjoin(buffered_game_grid, game_grid.to_crs('epsg:3857'), predicate='overlaps')

In [ ]:
neighboring_gdf

## Create network representation

In [ ]:
transport_network = networkx.DiGraph()
transport_network.add_nodes_from(game_grid['MGRS']) #add nodes, 1 for every entry of MGRS
# calculate travel time as an edge feature
neighboring_gdf['travel_time'] = 20/ (neighboring_gdf['transport_score_left'] + neighboring_gdf['transport_score_right'])
# create edges from (origin, destination, travel_time)
transport_network.add_weighted_edges_from(zip(neighboring_gdf['MGRS_left'], neighboring_gdf['MGRS_right'], neighboring_gdf['travel_time']))

In [ ]:
networkx.write_graphml(transport_network, 'transport_network_pre.graphml')

In [ ]:
transport_network_loaded = networkx.read_graphml('transport_network_pre.graphml')

## Visualizing the transport score
Highways and major roads have highest transport score. Rural areas have lowest transport score. Cities have intermediate transport score to model local roads

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,10))
game_grid.plot(column='transport_score', ax=ax)
plt.show()

In [ ]:
# plotting a route
def plot_route(game_grid, transport_network, route, ax, buffer=0.01, color=None):
    '''
    plots a route on the game_grid network
    
    game_grid: gdf of the game grid
    transport_network: networkx object representing network
    route: list of MGRS ID's of shelters in order of the route
    buffer: how much to buffer the routes for visibility
    ax: matplotlib axes object to plot on
    color: color of the route to plot
    '''
    if color == None:
        color = np.random.random(3)
    full_route = []
    for idx in range(len(route)-1):
        orig = route[idx]
        dest = route[idx+1]
        full_route = full_route + networkx.astar_path(transport_network, orig, dest)
    game_grid.set_index('MGRS').loc[full_route].buffer(buffer).plot(ax=ax, color=color)

In [ ]:
# example of plotting transport score and route
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1)
game_grid.plot(column='transport_score', cmap='Greys', alpha=0.8, ax=ax)
#plotting for random origin and destination for reference
plot_route(game_grid,
           transport_network, 
           [np.random.choice(game_grid['MGRS']),
            np.random.choice(game_grid['MGRS'])],
           ax=ax,
           buffer=0.01)
ctx.add_basemap(ax,
                crs=game_grid.crs,
                source=ctx.providers.Stamen.TonerBackground)
ax.set_xlim([w,e]) # from overall bounds
ax.set_ylim([s,n])
plt.show()